In [ ]:
import numpy as np
from random import randrange, randint, uniform, random 
import math

# initializations
dt = 0.1
T = 10
X_anchor = -2
Y_anchor = -2

# initializations
X = np.zeros((2,int(T/dt)))
Y = np.zeros((3,int(T/dt)))
R = np.zeros((2,2*int(T/dt))) # 2x2
Q = np.zeros((3,3*int(T/dt))) # 3x3
Vel = np.zeros((2,int(T/dt)))
nu = np.zeros((2,int(T/dt)))
nup = np.zeros((2,int(T/dt)))
Gt = [[1, 0], [0, 1]]
g = np.zeros((2,int(T/dt)))
h = np.zeros((3,int(T/dt)))
hy = np.zeros((3,int(T/dt)))
Ht = np.zeros((3,2))
Sp = np.zeros((2,2*int(T/dt)))
S = np.zeros((2,2*int(T/dt)))
K = np.zeros((2,3*int(T/dt)))
# Drone start position
# X[:,[0]] = [[x],[y]]

for t in range(1,T+1):
  # Control inputs
  Vel[0,[t]] = -5+random()*10 
  Vel[1,[t]] = -5+random()*10
  # Motion Model Update 
  R[0,[2*t-1]] = random()*0.04
  R[1,[2*t]] = random()*0.04
  nu[:,[t]] = X[0,[t-1]] + Vel[:,[t]]*dt
  X[0,[t]] = nu[0,[t-1]] + R[0,[2*t-1]] 
  X[1,[t]] = nu[0,[t-1]] + R[1,[2*t]] 
  # Measurement Model
  Q[:,[3*t-1, 3*t, 3*t+1]] = random()*0.04
  Y[0,[t]] = X[0,[t]] + Q[0,[3*t-1]] 
  Y[1,[t]] = X[1,[t]] + Q[1,[3*t]] 
  Y[2,[t]] = math.sqrt(math.pow((X[0,[t]]-X_anchor),2)+math.pow((X[1,[t]]-Y_anchor),2)) + Q[2,[3*t+1]]
  
  # Linearized Motion Model
  g[:,[t]] = nu[:,[t]] + np.matmul(Gt, X[:,[t-1]]-nu[:,[t-1]])
  
  # Linearized Measurement Model
  Ht = [[0, 1], [1, 0], [float((X[0,[t]]-X_anchor)/math.sqrt(pow(X[0,[t]]-X_anchor,2)+pow(X[1,[t]]-Y_anchor,2))), float((X[1,[t]]-Y_anchor)/math.sqrt(pow(X[0,[t]]-X_anchor,2)+pow(X[1,[t]]-Y_anchor,2)))]]
  hy[0,[t]] = nup[0,[t]]
  hy[1,[t]] = nup[1,[t]]
  hy[2,[t]] = math.sqrt(math.pow((nup[0,[t]]-X_anchor),2)+math.pow((nup[1,[t]]-Y_anchor),2))
  h[:,[t]] = hy[:,[t]] + np.matmul(Ht, X[:,[t]]-nu[:,[t]])
  # print(Ht[2][1]*2) 
  # Prediction Update
  nup[:,[t]] = np.matmul(Gt, nu[:,[t]])
  Sp[:,[2*t-1,2*t]] = np.matmul(Gt,np.matmul(S[:,[2*t-1,2*t]], np.transpose(Gt)))+ R[:,[2*t-1,2*t]] 
  # Measurement Update
  K[:,[3*t-1,3*t,3*t+1]] = np.matmul(np.matmul(Sp[:,[2*t-1,2*t]], np.transpose(Ht)), np.linalg.inv(np.matmul(Ht, np.matmul(Sp[:,[2*t-1,2*t]], np.transpose(Ht))) + Q[:,[3*t-1, 3*t, 3*t+1]]))
  nu[:,[t]] = nup[:,[t]] + np.matmul(K[:,[3*t-1,3*t,3*t+1]], Y[:,[t]]-h[:,[t]] )
  S[:,[2*t-1,2*t]] = np.matmul((np.identity(2) - np.matmul(K[:,[3*t-1,3*t,3*t+1]], Ht)),Sp[:,[2*t-1,2*t]])


print(S[:,[2*5-1,2*5]])
#print(np.shape(S))

  




[[1, 0], [0, 1]]
(2, 1)
[[ 2.97348647e-17  5.47040844e-18]
 [-6.06624736e-17 -6.72919033e-17]] yeeyyy
